In [3]:
# only for gated models 
from huggingface_hub import login
login()

In [6]:
import sys
import os
import subprocess

print("Python executable:", sys.executable)
print("Python prefix:", sys.prefix)
print("Conda environment:", os.environ.get('CONDA_DEFAULT_ENV'))
print("Conda prefix:", os.environ.get('CONDA_PREFIX'))

# Check pip location
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Pip location:", result.stdout.strip())

# Check pip version
result = subprocess.run(['pip', '--version'], capture_output=True, text=True)
print("Pip version:", result.stdout.strip())

Python executable: /home/sagemaker-user/.conda/envs/inception/bin/python
Python prefix: /home/sagemaker-user/.conda/envs/inception
Conda environment: base
Conda prefix: /opt/conda
Pip location: /home/sagemaker-user/.conda/envs/inception/bin/pip
Pip version: pip 25.2 from /home/sagemaker-user/.conda/envs/inception/lib/python3.9/site-packages/pip (python 3.9)


In [5]:
import os
import sys

# Add environment's bin directory to PATH
env_bin = os.path.dirname(sys.executable)
current_path = os.environ['PATH']
if env_bin not in current_path:
    os.environ['PATH'] = f"{env_bin}:{current_path}"

# Verify fix
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Fixed pip location:", result.stdout.strip())

Fixed pip location: /home/sagemaker-user/.conda/envs/inception/bin/pip


In [7]:
pip install --upgrade transformers accelerate bitsandbytes torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,  # Threshold for outlier detection
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offloading if needed
)

# Load the model and tokenizer
model_path = "inceptionai/jais-13b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print(f"Model memory footprint: {model.get_memory_footprint() / 1024**3:.2f} GB")


tokenizer_config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

configuration_jais.py:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

...

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model memory footprint: 12.69 GB


In [19]:
# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Prepare prompt
test_prompt = "What is artificial intelligence?"
inputs = tokenizer(test_prompt, return_tensors="pt")

# **Move inputs to the model's device**
input_ids = inputs.input_ids.to(device)

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=50,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Input:", test_prompt)
print("Output:", response)


Input: What is artificial intelligence?
Output: What is artificial intelligence?

Artificial intelligence (AI) is the science of making computers do things that would require intelligence if done by humans. AI is used in many fields including business, entertainment, and science.


In [21]:
# JAIS chat prompt
prompt = """### Instruction: Your name is Swapnil, and you are named after Jebel Jais.

### Input: [|Human|] Hello, can you introduce yourself?
### Response: [|AI|]"""

inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device)  # Move to GPU if available

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=100,
        temperature=0.3,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response.split("### Response: [|AI|]")[-1].strip())


Hi, I'm Swapnil. I'm named after Jebel Jais, the highest mountain in the United Arab Emirates.


In [22]:
# Print bitsandbytes memory footprint in GB
mem_bytes = model.get_memory_footprint()
print(f"Quantized model footprint: {mem_bytes / 1024**3:.2f} GB")


Quantized model footprint: 12.69 GB


In [23]:
model_name = "jais-13b-chat-8bit"               #  new repo name
save_dir   = f"./{model_name}"

# Save model and tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

RuntimeError: The weights trying to be saved contained shared tensors [{'lm_head.weight', 'transformer.wte.weight'}] that are mismatching the transformers base configuration. Try saving using `safe_serialization=False`, setting the `_dynamic_tied_weights_keys` attribute for affected modules, or remove this tensor sharing.

In [24]:
# When saving your quantized model, set safe_serialization=False
save_dir = "./jais-13b-chat-8bit-ss"

# Save model (disable safe serialization to avoid shared-tensor errors)
model.save_pretrained(save_dir, safe_serialization=False)
tokenizer.save_pretrained(save_dir)


('./jais-13b-chat-8bit-ss/tokenizer_config.json',
 './jais-13b-chat-8bit-ss/special_tokens_map.json',
 './jais-13b-chat-8bit-ss/tokenizer.json')

In [ ]:
import os
from huggingface_hub import Repository

# Navigate to your model directory and initialize git
os.chdir("./jais-13b-chat-8bit-ss")
os.system("git init")
os.system("git remote add origin https://huggingface.co/FilledVaccum/jais-13b-chat-8bit-ss")

# Then create Repository object
repo = Repository(local_dir=".", clone_from=None, token=token)
repo.push_to_hub(commit_message="Initial commit: 8-bit quantized JAIS model")

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
/home/sagemaker-user/.conda/envs/inception/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, Fu

Initialized empty Git repository in /home/sagemaker-user/Inception/jais-13b-chat-8bit-ss/.git/


TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true

In [34]:
from huggingface_hub import HfApi

api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="/home/sagemaker-user/Inception/jais-13b-chat-8bit-ss",
    repo_id="FilledVaccum/jais-13b-chat-8bit-ss",
    repo_type="model",
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/FilledVaccum/jais-13b-chat-8bit-ss/commit/d3ca85f5d072b48496c91fec74cd78c18da7d2b7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d3ca85f5d072b48496c91fec74cd78c18da7d2b7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/FilledVaccum/jais-13b-chat-8bit-ss', endpoint='https://huggingface.co', repo_type='model', repo_id='FilledVaccum/jais-13b-chat-8bit-ss'), pr_revision=None, pr_num=None)

In [35]:
from huggingface_hub import HfApi

api = HfApi(token=os.getenv("HF_TOKEN"))
files = api.list_repo_files(repo_id="FilledVaccum/jais-13b-chat-8bit-ss", repo_type="model")
print("Files in repo:", files)


Files in repo: ['.gitattributes', 'config.json', 'configuration_jais.py', 'generation_config.json', 'modeling_jais.py', 'pytorch_model-00001-of-00003.bin', 'pytorch_model-00002-of-00003.bin', 'pytorch_model-00003-of-00003.bin', 'pytorch_model.bin.index.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json']


In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

repo_id = "FilledVaccum/jais-13b-chat-8bit-ss"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(repo_id)
model     = AutoModelForCausalLM.from_pretrained(
    repo_id,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

print("Loaded model memory footprint:", model.get_memory_footprint() / 1024**3, "GB")


tokenizer_config.json:   0%|          | 0.00/484 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration_jais.py: 0.00B [00:00, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


Loaded model memory footprint: 12.690843045711517 GB


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

repo_id = "FilledVaccum/jais-13b-chat-8bit-ss"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load
tokenizer = AutoTokenizer.from_pretrained(repo_id)
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

# Prepare input and move it to the same device as the model
device = next(model.parameters()).device
inputs = tokenizer("Hello, Jais! What can you do?", return_tensors="pt")
input_ids = inputs.input_ids.to(device)

# Generate
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=50,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False
    )

# Decode
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


NotImplementedError: Cannot copy out of meta tensor; no data!